In [ ]:
%%capture
#Installing the requirements that Google Colab doesn't have
!pip install timm 
!pip install wandb --quiet
!pip install pytorch-lightning

In [ ]:
#Attaching our drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Bringing in the MODIS data
!cp '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MOD10A1_sierras.zip' .
!cp '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MYD10A1_sierras.zip' .
!cp '/content/drive/MyDrive/snowcapstone team spring 2022/Copernicus_Data/copernicus_sierras2.zip' .

#Unzipping it into the current folder
!unzip -qq MOD10A1_sierras.zip -d .
!unzip -qq MYD10A1_sierras.zip -d .
!unzip -qq copernicus_sierras2.zip -d .

In [ ]:
#All of our imports
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR

import torchvision
from torchvision import transforms as T
from torchvision.io import read_image

import timm

from tqdm import tqdm_notebook as tqdm

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import progress
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor

import sklearn
from sklearn.model_selection import StratifiedKFold

In [ ]:
#Checking out our list of trained models to choose from.
timm.list_models(pretrained=True)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [ ]:
class args:
  #Overall Args
  folder_name = "drive/MyDrive/snowcapstone team spring 2022/Modeling/"
  
  #Setting the number of CPU workers we are using
  num_workers = 4

  #Setting the seed so we can replicate
  seed = 1212

  #Toggle for whether or not we want our model pretrained on imagenet
  pretrained = True

  #Next we pick the model name with the appropriate shape, img size and output
  #model_name1 = 'tf_efficientnet_b3_ns'
  #model_shape1 = 1536
  model_name2 = 'tf_efficientnet_b6_ns'
  model_shape2 = 2304 #768 for swin small 1536 for swin large 1792 for efficientnet b4 768 for cait-m-36
  imagesize = 224
  num_classes = 1

  #Training Args
  train_batch_size = 32
  val_batch_size = 32
  test_batch_size = 32

  #Max epochs and number of folds
  max_epochs = 75
  n_splits = 2
  
  #Optimizer and Scheduler args
  loss = 'nn.BCEWithLogitsLoss'
  lr = 1e-4
  warmup_epochs = 0
  weight_decay = 1e-6
  eta_min = 0.0000001
  n_accumulate = 1
  T_0 = 25
  T_max = 100

  #Callback args
  #Minimum number amount of improvement to not trigger patience
  min_delta = 0.0
  #Number of epochs in a row to wait for improvement
  patience = 75

#Dataloader Args
loaderargs = {'num_workers' : args.num_workers, 'pin_memory': False, 'drop_last': False}
device = torch.device("cuda:0")

seed_everything(args.seed)


Global seed set to 1212


1212

In [ ]:
#Reading in the data
df = pd.read_csv(f'/content/{args.folder_name}traindf.csv').drop('Unnamed: 0', axis = 1)

#Designating which columns are our metadata
feature_cols = [col for col in df.columns if col not in ['cell_id', 'date', 'MOD10A1_filelocations', 'MYD10A1_filelocations', 'copernicus_filelocations', 'SWE']]

#Min max scaling the meta data
scaler = sklearn.preprocessing.MinMaxScaler()
df[feature_cols] =  scaler.fit_transform(df[feature_cols])

#We will create a separate scaler for the targets so that we can transform them back and forth
target_scaler = sklearn.preprocessing.MinMaxScaler()
target_scaler.fit(np.array(df['SWE']).reshape(-1, 1))
df['SWE_Scaled'] = target_scaler.transform(np.array(df['SWE']).reshape(-1, 1))

tabluar_columns = len(feature_cols)

In [ ]:
#Updating the modis file locations column to the correct location
df['MOD10A1_filelocations'] = df['MOD10A1_filelocations'].str.replace('/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MOD10A1/', '/content/MOD10A1/')
df['MYD10A1_filelocations'] = df['MYD10A1_filelocations'].str.replace('/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MYD10A1/', '/content/MYD10A1/')
df['copernicus_filelocations'] = df['copernicus_filelocations'].str.replace('/content/drive/MyDrive/snowcapstone team spring 2022/Copernicus_Data/', '/content/Copernicus_Data/')

In [ ]:
#Making sure we don't have any any NAs 
print(feature_cols)
print(df.isna().sum())
df = df.fillna(0)
print(df.isna().sum())

['mean_inversed_swe', 'mean_local_swe', 'median_local_swe', 'max_local_swe', 'min_local_swe', 'mean_local_elevation', 'median_local_elevation', 'max_local_elevation', 'min_local_elevation']
cell_id                     0
date                        0
SWE                         0
mean_inversed_swe           0
mean_local_swe              0
median_local_swe            0
max_local_swe               0
min_local_swe               0
mean_local_elevation        0
median_local_elevation      0
max_local_elevation         0
min_local_elevation         0
MOD10A1_filelocations       0
MYD10A1_filelocations       0
copernicus_filelocations    0
SWE_Scaled                  0
dtype: int64
cell_id                     0
date                        0
SWE                         0
mean_inversed_swe           0
mean_local_swe              0
median_local_swe            0
max_local_swe               0
min_local_swe               0
mean_local_elevation        0
median_local_elevation      0
max_local_elevati

In [ ]:
#Datasets are how pytorch knows how to read in the data
class SWEDataset(torch.utils.data.Dataset):
    def __init__(self, df, test = False):
        self.df = df
        #First we must specify the path to the images
        self.MOD10A1_file_names = df['MOD10A1_filelocations'].values
        self.MYD10A1_file_names = df['MYD10A1_filelocations'].values
        self.copernicus_file_names = df['copernicus_filelocations'].values
        #The only transform we want to do right now is the resizing
        self._transform = T.Resize(size= (args.imagesize, args.imagesize))
        #We specify the tabular feature columns
        self.meta = df[feature_cols].values
        #Now we specify the targets
        self.targets = df['SWE_Scaled'].values
        #Finally we specify if this is training or test
        self.test = test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        #Get the image, scale it to between 0-1 and resize it
        MOD10A1_img_path = self.MOD10A1_file_names[index]
        MOD10A1_img = read_image(MOD10A1_img_path, mode = torchvision.io.ImageReadMode.RGB) / 255
        MOD10A1_img = self._transform(MOD10A1_img)

        MYD10A1_img_path = self.MYD10A1_file_names[index]
        MYD10A1_img = read_image(MYD10A1_img_path, mode = torchvision.io.ImageReadMode.RGB) / 255
        MYD10A1_img = self._transform(MYD10A1_img)

        copernicus_img_path = self.copernicus_file_names[index]
        copernicus_img = read_image(copernicus_img_path, mode = torchvision.io.ImageReadMode.RGB) / 255
        copernicus_img = self._transform(copernicus_img)


        #Pull in the features for our batch
        meta = self.meta[index, :]
        
        #Specify the target based on whether this is training or test
        if self.test:
          target = 0
        else:
          target = self.targets[index]
            
        return MOD10A1_img, MYD10A1_img, copernicus_img, target, meta

In [ ]:
#Pytorch Lightning Requires that the dataset be formatted as a module
class SWEDataModule(pl.LightningDataModule):
    def __init__(self, traindf, valdf,args, loaderargs):
        super().__init__()
        #Import our training and validation set, which we will define later
        self._train_df = traindf
        self._val_df = valdf

        #Makesure we bring in our args so we can use them
        self.args = args
        self.loaderargs = loaderargs

    #Building the datasets
    def __create_dataset(self, train=True):
        if train == 'train':
          return SWEDataset(self._train_df)
        else:
          return SWEDataset(self._val_df)

    #Using the datasets to return a dataloader
    def train_dataloader(self):
        SWE_train = self.__create_dataset("train")
        return DataLoader(SWE_train, **self.loaderargs, batch_size=self.args.train_batch_size)

    def val_dataloader(self):
        SWE_val = self.__create_dataset("val")
        return DataLoader(SWE_val, **self.loaderargs, batch_size=self.args.val_batch_size)
    

In [ ]:
def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                #T.RandomHorizontalFlip(),
                #T.RandomVerticalFlip(),
                #T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean = (0.485, 0.456, 0.406), 
                            std = (0.229, 0.224, 0.225))
                
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean = (0.485, 0.456, 0.406), 
                            std = (0.229, 0.224, 0.225))
            ]
        ),
    }
    return transform
  

def mixup(x1: torch.Tensor, y: torch.Tensor, 
          z = torch.Tensor, alpha: float = 1.0):
    assert alpha > 0, "alpha should be larger than 0"
    assert x1.size(0) > 1, "Mixup cannot be applied to a single instance."

    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x1.size()[0])
    mixed_x1 = lam * x1 + (1 - lam) * x1[rand_index, :]
    mixed_meta = lam * z + (1 - lam) * z[rand_index, :]
    target_a, target_b = y, y[rand_index]
    return mixed_x1, mixed_meta, target_a, target_b,  lam

In [ ]:
#Use this to find the model_shape attribute when changing models

x = torch.randn(1,9,args.imagesize,args.imagesize)
model = timm.create_model(args.model_name2, #                                       pretrained=args.pretrained, 
                                      num_classes=0,
                                      in_chans = 9)
model(x).shape

torch.Size([1, 2304])

In [ ]:
class SWEModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.args = args
        self.scaler = target_scaler
        self.tabular_columns = tabluar_columns
        #Image Models
        self.model1 = timm.create_model(args.model_name2, 
                                       pretrained=args.pretrained, 
                                       num_classes=0,
                                       in_chans = 9)

        self.fc1 = nn.Linear(args.model_shape2, 768)
        self.fc2 = nn.Linear(768 + self.tabular_columns, 384)
        self.fc3 = nn.Linear(384, 96)
        self.fc4 = nn.Linear(96, args.num_classes)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU()
        self._criterion = eval(self.args.loss)()
        self.transform = get_default_transforms()

        #Additional convolution stuff (only if needed)
        #self.cv1 = nn.Conv2d(args.model_shape2, 256 ,kernel_size=3,stride=1,padding=1)
        #self.bn1 = nn.BatchNorm2d(256)
        #self.cv2 = nn.Conv2d(256, 64 ,kernel_size=3,stride=1,padding=1)
        #self.bn2 = nn.BatchNorm2d(64)
        #self.mp1 = nn.MaxPool2d(kernel_size=1,stride=1)
        #self.mp2 = nn.MaxPool2d(kernel_size=2,stride=2)

    def forward(self, features1, meta):
        #Image Models
        features1 = self.model1(features1)                 
        features1 = self.relu(features1)

        
        #First Linear Model
        features = self.fc1(features)
        features = self.relu(features)

        #Convolution without pooling (output of image model is non-pooled tensor)
        #features = self.cv1(features)
        #features = self.bn1(features)
        #features = self.relu(features)
        #features = self.dropout(features)
        #features = self.mp1(features)
        #features = self.cv2(features)
        #features = self.bn2(features)
        #features = self.relu(features)
        #features = self.mp2(features)

        #Concatenating the meta data
        features = torch.cat([features, meta], dim=1)
        
        #Final fully connected layers
        features = self.fc2(features)
        features = self.relu(features)
        
        features = self.fc3(features)
        features = self.relu(features)
        
        output = self.fc4(features)           
        return output


    def __share_step(self, batch, mode):
        MOD10A1_img, MYD10A1_img, copernicus_img, labels, meta = batch
        labels = labels.float()
        meta = meta.float()
        MOD10A1_img = self.transform[mode](MOD10A1_img)
        MYD10A1_img = self.transform[mode](MYD10A1_img)
        copernicus_img = self.transform[mode](copernicus_img)

        img = torch.cat((MOD10A1_img, MYD10A1_img, copernicus_img), axis = 1)

        rand_index = torch.rand(1)[0]
        
        #This is a mixup function
        if rand_index < 0.5 and mode == 'train':
            mixed_img, mixed_meta, target_a, target_b, lam = mixup(img, 
                                                          labels, meta, alpha=0.5)
            logits = self.forward(mixed_img, mixed_meta).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + \
                (1 - lam) * self._criterion(logits, target_b)

        else:  
          logits = self.forward(img, meta).squeeze(1)
          loss = self._criterion(logits, labels)

        pred = torch.from_numpy(self.scaler \
            .inverse_transform(np.array(logits.sigmoid().detach().cpu()) \
            .reshape(-1, 1)))
        labels = torch.from_numpy(self.scaler \
            .inverse_transform(np.array(labels.detach().cpu()) \
            .reshape(-1, 1)))
        
        '''
        #This is random noise
        elif rand_index > 0.8 and mode == 'train':
            images = images + (torch.randn(images.size(0),3,args.imagesize,args.imagesize, 
                                           dtype = torch.float, device = device)*10)/100
            logits = self.forward(images, meta).squeeze(1)
            loss = self._criterion(logits, labels)
        '''

        return loss, pred, labels

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        return {'loss': loss, 'pred': pred, 'labels': labels}



    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return {'pred': pred, 'labels': labels}


    def training_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'train')

    def validation_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'val')

        
    def __share_epoch_end(self, outputs, mode):
        preds = []
        labels = []
        for out in outputs:
            pred, label = out['pred'], out['labels']
            preds.append(pred)
            labels.append(label)
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        metrics = torch.sqrt(((labels - preds) ** 2).mean())
        self.log(f'{mode}_RMSE', metrics)    


    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=args.lr, weight_decay = args.weight_decay)
        
        return {
        "optimizer": optimizer,
        #"lr_scheduler": {
        #    "scheduler": CosineAnnealingLR(optimizer, T_max = args.T_max, eta_min= args.eta_min),
        #    "interval": "step",
        #    "monitor": "train_loss",
        #    "frequency": 1}
            }

In [ ]:
Kfolds = StratifiedKFold(n_splits=args.n_splits, shuffle=True, 
                         random_state = args.seed)

num_bins = int(np.ceil(2*((len(df))**(1./3))))

df['bins'] = pd.cut(df['SWE'], bins=num_bins, labels=False)


for fold, (train_idx, val_idx) in enumerate(Kfolds.split(df["cell_id"], df["bins"])):
    traindf = df.loc[train_idx].reset_index(drop=True)
    valdf = df.loc[val_idx].reset_index(drop=True)

    model = SWEModel()

    #Callbacks
    early_stop_callback = EarlyStopping(monitor="val_RMSE", min_delta=args.min_delta, patience=args.patience, 
                                        verbose=False, mode="min")
    progressbar = TQDMProgressBar(refresh_rate = 10)
    checkpoint_callback = ModelCheckpoint(dirpath=args.folder_name, 
                                          filename= f"{fold}best_weights", save_top_k=1, monitor="val_RMSE")
    lr_monitor = LearningRateMonitor(logging_interval='step')

    wandb_logger = WandbLogger(project="Snowcast", entity="snowcastshowdown", job_type='train', log_model = 'all')

    wandb_logger.watch(model)

    trainer = pl.Trainer(max_epochs=args.max_epochs, 
                        gpus=1, 
                        logger=wandb_logger,
                        callbacks=[early_stop_callback, 
                                    progressbar, 
                                    checkpoint_callback,
                                    lr_monitor])

    SWE_Datamodule = SWEDataModule(traindf, valdf, args = args, loaderargs = loaderargs)

    trainer.fit(model, SWE_Datamodule)

    wandb.finish()
    
    del model
    torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,
wandb: Currently logged in as: malachyiii (use `wandb login --relogin` to force relogin)


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | model1     | EfficientNet      | 40.7 M
1 | fc1        | Linear            | 1.8 M 
2 | fc2        | Linear            | 298 K 
3 | fc3        | Linear            | 37.0 K
4 | fc4        | Linear            | 97    
5 | dropout    | Dropout           | 0     
6 | relu       | ReLU              | 0     
7 | _criterion | BCEWithLogitsLoss | 0     
-------------------------------------------------
42.8 M    Trainable params
0         Non-trainable params
42.8 M    Total params
171.379   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/drive/.shortcut-tar

Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 1212


Training: 0it [00:00, ?it/s]